In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from NeutralNet import NeuralNet
import os

# 判定GPU是否存在

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义超参数
input_size = 784
hidden_size = 20
num_classes = 10
num_epochs = 1
batch_size = 100
learning_rate = 0.1

# 数据集dir
data_dir = 'resources/dataset'

In [11]:
# 载入手写体数据 如果不存在则下载 注意科学上网，否则下载很慢 
train_dataset = torchvision.datasets.MNIST(root=data_dir,
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root=data_dir,
                                          train=False,
                                          transform=transforms.ToTensor())
print("download dataset successfully!")

download dataset successfully!


In [12]:
# 构建数据管道
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# 定义模型
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# 损失函数和优化算法
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# , weight_decay=1e-8
# 初始化权重和偏置
for name,param in model.named_parameters():
        if 'weight' in name:
            nn.init.kaiming_uniform_(param, nonlinearity='relu')
        if 'bias' in name:
            nn.init.zeros_(param)

print(next(model.parameters()).is_cuda)
print("build model successfully!")

True
build model successfully!


In [13]:
# 训练模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # 前向传播和计算loss
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 后向传播和调整参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 每100个batch打印一次数据
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3574
Epoch [1/5], Step [200/600], Loss: 0.2406
Epoch [1/5], Step [300/600], Loss: 0.3087
Epoch [1/5], Step [400/600], Loss: 0.1526
Epoch [1/5], Step [500/600], Loss: 0.2370
Epoch [1/5], Step [600/600], Loss: 0.3277
Epoch [2/5], Step [100/600], Loss: 0.1337
Epoch [2/5], Step [200/600], Loss: 0.0961
Epoch [2/5], Step [300/600], Loss: 0.1237
Epoch [2/5], Step [400/600], Loss: 0.1275
Epoch [2/5], Step [500/600], Loss: 0.0859
Epoch [2/5], Step [600/600], Loss: 0.0886
Epoch [3/5], Step [100/600], Loss: 0.0735
Epoch [3/5], Step [200/600], Loss: 0.0834
Epoch [3/5], Step [300/600], Loss: 0.1071
Epoch [3/5], Step [400/600], Loss: 0.0939
Epoch [3/5], Step [500/600], Loss: 0.0720
Epoch [3/5], Step [600/600], Loss: 0.0985
Epoch [4/5], Step [100/600], Loss: 0.0670
Epoch [4/5], Step [200/600], Loss: 0.0179
Epoch [4/5], Step [300/600], Loss: 0.0486
Epoch [4/5], Step [400/600], Loss: 0.0573
Epoch [4/5], Step [500/600], Loss: 0.0316
Epoch [4/5], Step [600/600], Loss:

In [14]:
# 模型测试部分
# 测试阶段不需要计算梯度，注意
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print(_, predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.86 %


In [15]:
flag = torch.cuda.is_available()
device_cnt = torch.cuda.device_count()
device_name=torch.cuda.get_device_name(0)
device_curr = torch.cuda.current_device()

print(flag)
print(device_cnt)
print(device_name)
print(device_curr)

True
1
GeForce GTX 1050 Ti
0
